# Complete CNN Pipeline
(asumes CNN models are already created)

 1. Primary CNN prediction
 1. Post-processing with secondary CNN
 1. Morphological post-processing
 1. Converting to submission
 

In [1]:
from helpers.dataset_preprocessing import create_dataset, extract_patches, compute_input_features, compute_output_features,load_image,img_crop_translate
from helpers.dataset_postprocessing import unpatch
from helpers.feature_extractors import extract_features, extract_features_2d, value_to_class, value_to_2d_class, img_to_label
from helpers.visualization_helpers import *
from helpers.CNN_helpers import *

from skimage.util import img_as_ubyte
from skimage import io
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk, square
from keras.models import load_model


import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import numpy as np

%matplotlib inline

Using TensorFlow backend.


### Global Variables

In [2]:
PATCH_SIZE = 8
PATCH_TRANSLATION = 0
FOREGROUND_THRESHOLD = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
ORIGINAL_IMAGE_WIDTH = 400
ORIGINAL_IMAGE_HEIGHT = 400
NUM_CHANNELS = 3

## 1. Primary CNN prediction

In [3]:
#  Load keras primary CNN model
primary_model = load_model('models/ec2_model.h5')

In [4]:
predict_imgs(model=primary_model, data_dir='test_images/', prediction_root_dir='test_images_primary_prediction/', no_of_imgs=50, img_name='test')

Predicting test_images/test_001.png
Predicting test_images/test_002.png
Predicting test_images/test_003.png
Predicting test_images/test_004.png
Predicting test_images/test_005.png
Predicting test_images/test_006.png
Predicting test_images/test_007.png
Predicting test_images/test_008.png
Predicting test_images/test_009.png
Predicting test_images/test_010.png
Predicting test_images/test_011.png
Predicting test_images/test_012.png
Predicting test_images/test_013.png
Predicting test_images/test_014.png
Predicting test_images/test_015.png
Predicting test_images/test_016.png
Predicting test_images/test_017.png
Predicting test_images/test_018.png
Predicting test_images/test_019.png
Predicting test_images/test_020.png
Predicting test_images/test_021.png
Predicting test_images/test_022.png
Predicting test_images/test_023.png
Predicting test_images/test_024.png
Predicting test_images/test_025.png
Predicting test_images/test_026.png
Predicting test_images/test_027.png
Predicting test_images/test_

## 2. Post-processing with secondary CNN

In [5]:
#  Load keras secondary post processing CNN model
secondary_model = load_model('models/post_CNN.h5')

In [6]:
predict_imgs(model=secondary_model, data_dir='test_images_primary_prediction/', prediction_root_dir='test_images_secondary_prediction/', no_of_imgs=50, img_name='prediction')

Predicting test_images_primary_prediction/prediction_001.png
Predicting test_images_primary_prediction/prediction_002.png
Predicting test_images_primary_prediction/prediction_003.png
Predicting test_images_primary_prediction/prediction_004.png
Predicting test_images_primary_prediction/prediction_005.png
Predicting test_images_primary_prediction/prediction_006.png
Predicting test_images_primary_prediction/prediction_007.png
Predicting test_images_primary_prediction/prediction_008.png
Predicting test_images_primary_prediction/prediction_009.png
Predicting test_images_primary_prediction/prediction_010.png
Predicting test_images_primary_prediction/prediction_011.png
Predicting test_images_primary_prediction/prediction_012.png
Predicting test_images_primary_prediction/prediction_013.png
Predicting test_images_primary_prediction/prediction_014.png
Predicting test_images_primary_prediction/prediction_015.png
Predicting test_images_primary_prediction/prediction_016.png
Predicting test_images_p

In [7]:
data_dir = 'test_images_primary_prediction/'
pred_dir = 'test_images_secondary_prediction/'

if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

    for i in range(1, 51):
        num = str(i).zfill(3)
        imageid = "prediction_" + num
        image_filename = data_dir + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Predicting ' + image_filename)
            # If using img_crop_translate:
            #img = mpimg.imread(image_filename)
            #data = np.asarray(img_crop(img, PATCH_SIZE, PATCH_SIZE))

            # If using img_crop_translate:
            pil_img = Image.open(image_filename)
            pil_img = pil_img.convert('RGB')
            img = np.array(pil_img)
            data = np.asarray(img_as_float(img_crop_translate(img, PATCH_SIZE, PATCH_SIZE, 0, 0)), dtype='float32')


            predictions_patch = secondary_model.predict_classes(data, verbose=1)

            img_prediction = label_to_img(img.shape[0], img.shape[1], 
                                              PATCH_SIZE, PATCH_SIZE, 
                                              predictions_patch)

            # If using img_crop_translate:
            #pimg = Image.fromarray((img_prediction*255.0).astype(np.uint8))
            #pimg = ImageOps.invert(pimg)


            # If using img_crop_translate:
            pimg = Image.fromarray((img_prediction*255.0).astype(np.uint8))
            
            pimg = ImageOps.invert(pimg)
            pimg = pimg.rotate(90)
            pimg = ImageOps.flip(pimg)

            pimg.save(pred_dir + "prediction_" + num + ".png")

        else:
            print ('File ' + image_filename + ' does not exist')
else:
    print("Prediction already made. If you want to rerun then delete old prediction directory.")

Prediction already made. If you want to rerun then delete old prediction directory.


## 3. Morphological post-processing

In [ ]:
data_dir = 'test_images_secondary_prediction/'
pred_dir = 'test_images_morphological_postprocessing/'

if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

    for i in range(1, 51):
        num = str(i).zfill(3)
        imageid = "prediction_" + num
        image_filename = data_dir + imageid + ".png"
        if os.path.isfile(image_filename):
            
            road = img_as_ubyte(io.imread(image_filename, as_grey=True))
            
            # Closing
            selem = square(9)
            closed = closing(road, selem)
            
            #Opening closed
            selem = square(9)
            open_closed = opening(closed, selem)
            
            #Closing opened-closed
            selem = square(35)
            closed_open_closed = closing(open_closed, selem)
            
            io.imsave(pred_dir + "prediction_" + num + ".png", closed_open_closed)
            
        else:
            print ('File ' + image_filename + ' does not exist')
else:
    print("Morphological post-processing already made. If you want to rerun then delete old prediction directory.")

Morphological post-processing already made. If you want to rerun then delete old prediction directory.


## 4. Converting to submission

In [ ]:
predictions_to_submission(prediction_root_dir='test_images_morphological_postprocessing/', patch_size=16)

test_images_morphological_postprocessing/prediction_001.png
test_images_morphological_postprocessing/prediction_002.png
test_images_morphological_postprocessing/prediction_003.png
test_images_morphological_postprocessing/prediction_004.png
test_images_morphological_postprocessing/prediction_005.png
test_images_morphological_postprocessing/prediction_006.png
test_images_morphological_postprocessing/prediction_007.png
test_images_morphological_postprocessing/prediction_008.png
test_images_morphological_postprocessing/prediction_009.png
test_images_morphological_postprocessing/prediction_010.png
test_images_morphological_postprocessing/prediction_011.png
test_images_morphological_postprocessing/prediction_012.png
test_images_morphological_postprocessing/prediction_013.png
test_images_morphological_postprocessing/prediction_014.png
test_images_morphological_postprocessing/prediction_015.png
test_images_morphological_postprocessing/prediction_016.png
test_images_morphological_postprocessing